# Visualization

## Overview of Notebook
This notebook presents to codes that generate specific types of geodata plots with anested barplot. They are meant to loop through geodata that has different values over a time index. The files are required to be in seperate files for each time index.

### 1) Import Modules and the Fores Fire Data
Weed need to import the following modules, and the fores fire data to plot the count of fire for each year as a reference.

In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import mplcursors

import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
from matplotlib.colors import Normalize


In [2]:
path = r"I:\CME538 Project\processed data ontario only\250m\Approximations\compiled_climated_files"

In [3]:
all_fires = pd.read_csv(r"I:\CME538 Project\processed data ontario only\250m\Approximations"+"\\"+"Ontario_Fires2001_2021.csv")
all_fires.rename(columns={'REP_DATE':'date'},inplace=True)
all_fires['date'] = pd.to_datetime(all_fires['date'])

#filter by year and month period
all_fires['year_month'] = all_fires['date'].dt.to_period('M')



In [4]:
#Groupby the year month, and count() totals.
all_fires_counted = all_fires.groupby('year_month')['FIRE_ID'].count().to_frame().rename(columns={'FIRE_ID':'count'}).reset_index()

In [5]:
all_fires_counted

,year_month,count
0,2001-01,1
1,2001-03,1
2,2001-04,52
3,2001-05,114
4,2001-06,267
...,...,...
173,2021-06,332
174,2021-07,571
175,2021-08,123
176,2021-09,15


### 2) Individual 3D Plot
This code generates a 3D space plot with a set variable that is plotted as vertical lines on a 2D map. A nest subplot is given also. 

In [ ]:
from matplotlib.colors import TwoSlopeNorm, Normalize
# Select pixel or grid size. This will reduce our data to a more manageable size
#by taking the average in that grid
grid_size = 5000


for file in os.listdir(path):
    print(file)

    #read file into dataframe and convert to geopandas, convert to projected crs
    data = pd.read_parquet(path+"\\"+file,engine="pyarrow")
    data = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon,data.lat),crs = 4269)
    data.to_crs(3347,inplace=True)
    
    
    lat = data.geometry.y
    data.lon = lon
    data.lat = lat
    data.drop(columns='geometry',inplace=True)
    data = pd.DataFrame(data)
    print("done with opening, and reprojecting lat and lon to 3347")

    #Create the new grid based on the mean
    data['lon_grid'] = (data.lon//grid_size)*grid_size
    data['lat_grid'] = (data.lat//grid_size)*grid_size

    #filter out bad data
    data = data[data.age >= 0]

    #filter to grid data
    temp = data.groupby(['lat_grid','lon_grid']).mean().drop(columns=['lat','lon']).reset_index()
    temp.rename(columns={'lon_grid':'lon','lat_grid':'lat'},inplace=True)
    print(len(temp))
    print('done resampling file')

    #create temp geodataframe for plotting
    temp = gpd.GeoDataFrame(temp,geometry=gpd.points_from_xy(temp.lon,temp.lat),crs = 3347)
    fires = gpd.GeoDataFrame(data[['lon','lat','burned']][data.burned == 1],geometry=gpd.points_from_xy(data['lon'][data.burned == 1],data['lat'][data.burned == 1]),crs=3347)
    
    #create file name for use
    file_sep = file.split(".")[0].split("_")
    if int(file_sep[1]) < 10:
        year = file_sep[0]+"-0"+file_sep[1]
    else:
         year = file_sep[0]+"-"+file_sep[1]
    print(year)

    #the next part loops through four variables we want to plot, the names, titles and color maps
    #to use
    types = ['age','vol','Tm','P']
    titles = ["Approximate Age of Ontario's Forests in",
              "Approximate Volume of Ontario's Forests in",
              "Average Temperature in",
              "Total Precipitaion in Ontario in"]
    labels = ['Age (years)','Volume (m^3/ha)','Avg Temp (*C)','Total Precipitaion (mm)']
    mapcolors = ['viridis','viridis','coolwarm','cividis_r']
    for type,lab,title,mapcolor in zip(types,labels,titles,mapcolors):

        #Create the color map norm for each data type
        if mapcolor == 'coolwarm':
            norm = TwoSlopeNorm(vmin=-40, vcenter=0, vmax=30)
        elif mapcolor == 'cividis_r':
            if temp[type].max() < 250:
                norm = Normalize(0, 250)
            else:
                norm = Normalize(0,temp[type].max())
        else:
            if temp[type].max() < 175:
                norm = Normalize(0, 175)
            else:
                norm = Normalize(0,temp[type].max())
        
        #Create figure with subplots
        fig = plt.figure(figsize=(20,20))
        gs = gridspec.GridSpec(2,1,height_ratios=[8,1])
        ax0 = plt.subplot(gs[0], projection='3d')

        #plot the variable based on the lat and lon
        sc = ax0.scatter(temp['lon'], temp['lat'], 0,c=temp[type], cmap=mapcolor,marker='s', s=2,zorder=1,norm=norm)


        #plot an orange line if a fire occured (1.0)
        if (len(fires) != 0) & (file.split("_")[1] != '12'):
            ax0.scatter(fires['lon'], fires['lat'], color='orangered', s=1, label='Fire')
            fixed_z_value = 1  # Adjust the fixed z-value as needed
            for x, y in zip(fires['lon'], fires['lat']):
                ax0.plot([x, x], [y, y], [0, fixed_z_value], color='orangered', linestyle='-', linewidth=1,zorder=100)
        

        #Create the colorbar and set some fomratting
        norm0 = norm
        sm0 = plt.cm.ScalarMappable(cmap=mapcolor, norm=norm0)
        sm0.set_array([])
        cbar0 = fig.colorbar(sm0, ax=ax0, label=lab,shrink=0.6)
        cbar0.set_label(lab, fontsize=22)
        cbar_tick_labels = cbar0.ax.get_yticklabels()
        for label in cbar_tick_labels:
            label.set_fontsize(20)

        #Format the first figure
        ax0.set_box_aspect([1, 1, 0.05])
        ax0.set_title(title+" {}".format(year),fontsize=26)
        ax0.set_xlabel("Longitude (EPSG 3347, in 10^6 meters )",fontsize=22,labelpad=20)
        ax0.set_ylabel("Lattitude (EPSG 3347, in 10^6 meters)",fontsize=22,labelpad=20)
        ax0.legend(fontsize=22)
        ax0.tick_params(axis='both', which='major', labelsize=16)
        ax0.set_xlim([5800000, 7600000])
        ax0.set_ylim([600000, 2400000])
        ax0.set_zlim([0, 1])


        #Generate the second subplot, a date vs. count of fires per year
        ax1 = plt.subplot(gs[1])
        ax1.bar(all_fires_counted['year_month'].astype(str),all_fires_counted['count'])

        #Get the year and number of fires to plot in red
        fire_month = all_fires_counted[all_fires_counted['year_month'].astype(str)==year]
        print(file)
        print(fire_month)
        print(year)

        #Format sub figure
        ax1.tick_params(axis='both', which='major', labelsize=16)
        ax1.set_title("Forest Fire Counts Per Month from 2001 to 2021",fontsize=26)
        ax1.set_xlabel("Date",fontsize=22)
        ax1.set_ylabel("Count",fontsize=22)
        step = len(all_fires_counted)//20
        ax1.set_xticks(all_fires_counted.index[::step],all_fires_counted['year_month'].astype(str).iloc[::step],rotation=45)
        
        #Plot the red bar if there are fires
        if len(fire_month) != 0:
            ax1.bar(fire_month['year_month'].astype(str),fire_month['count'],color='red')
        ax0.view_init(elev=40, azim=-75) 
        
        #Save and show figures
        name = str((int(file.split("_")[0])%100)*12-12 + int(file.split("_")[1]))
        plt.savefig(r"I:\CME538 Project\processed data ontario only\250m\Approximations\\"+type+"\\"+name+"_"+type+".png",dpi = 300)
        plt.show()


### 3) Combined 2D Plot
This code creates a collage of all four 2D variable maps with a fire count subplot.

In [ ]:

from matplotlib.colors import Normalize, ListedColormap, TwoSlopeNorm
from matplotlib.cm import ScalarMappable

# Select pixel or grid size. This will reduce our data to a more manageable size
#by taking the average in that grid
grid_size = 5000

for file in os.listdir(path):
    print(file)
    #read file into dataframe and convert to geopandas, convert to projected crs
    data = pd.read_parquet(path+"\\"+file,engine="pyarrow")
    data = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon,data.lat),crs = 4269)
    data.to_crs(3347,inplace=True)
    lon = data.geometry.x
    lat = data.geometry.y
    
    data.lon = lon
    data.lat = lat
    data.drop(columns='geometry',inplace=True)
    data = pd.DataFrame(data)
    print("done with opening, and reprojecting lat and lon to 3347")

    #Create the new grid based on the mean
    data['lon_grid'] = (data.lon//grid_size)*grid_size
    data['lat_grid'] = (data.lat//grid_size)*grid_size

    #filter out bad data
    data = data[data.age >= 0]

    #filter to grid data
    temp = data.groupby(['lat_grid','lon_grid']).mean().drop(columns=['lat','lon']).reset_index()
    temp.rename(columns={'lon_grid':'lon','lat_grid':'lat'},inplace=True)
    print(len(temp))
    print('done resampling file')

    #create temp geodataframe for plotting
    temp = gpd.GeoDataFrame(temp,geometry=gpd.points_from_xy(temp.lon,temp.lat),crs = 3347)
    fires = gpd.GeoDataFrame(data[['lon','lat','burned']][data.burned == 1],geometry=gpd.points_from_xy(data['lon'][data.burned == 1],data['lat'][data.burned == 1]),crs=3347)
    harvest = gpd.GeoDataFrame(data[['lon','lat','harvested']][data.harvested == 1],geometry=gpd.points_from_xy(data['lon'][data.harvested == 1],data['lat'][data.harvested == 1]),crs=3347)
    
    #create file name for use
    file_sep = file.split(".")[0].split("_")
    if int(file_sep[1]) < 10:
        year = file_sep[0]+"-0"+file_sep[1]
    else:
         year = file_sep[0]+"-"+file_sep[1]
    print(year)


    types = ['age','vol','Tm','P']
    titles = ["Approximate Age of Ontario's Forests in",
              "Approximate Volume of Ontario's Forests in",
              "Average Temperature in",
              "Total Precipitaion in Ontario in"]
    labels = ['Age (years)','Volume (m^3/ha)','Avg Temp (*C)','Total Precipitaion (mm)']
    mapcolors = ['viridis','viridis','coolwarm','cividis_r']
    fig = plt.figure(figsize=(20, 10))
    gs = gridspec.GridSpec(2, 4, height_ratios=[8, 2])
    for i,type,lab,title,mapcolor in zip(range(4),types,labels,titles,mapcolors):
        if mapcolor == 'coolwarm':
            norm = TwoSlopeNorm(vmin=-40, vcenter=0, vmax=30)
        elif mapcolor == 'cividis_r':
            if temp[type].max() < 250:
                norm = Normalize(0, 250)
            else:
                norm = Normalize(0,temp[type].max())
        else:
            if temp[type].max() < 175:
                norm = Normalize(0, 175)
            else:
                norm = Normalize(0,temp[type].max())
        
        #Create figure with subplots
        ax = plt.subplot(gs[0, i])

        #plot the variable based on the lat and lon
        sc = ax.scatter(temp['lon'], temp['lat'],c=temp[type], cmap=mapcolor,marker='s', s=0.4,zorder=1,norm=norm)

        #plot an orange line if a fire occured (1.0) and if harvest occured (only in decembre)
        if (len(fires) != 0) & (file.split("_")[1] != '12'):
            ax.scatter(fires['lon'], fires['lat'], color='orangered', s=1, label='Fire')
        if (len(harvest) != 0) & (file.split("_")[1] == '12'):
            ax.scatter(harvest['lon'], harvest['lat'], color='black', s=1, label='Harvested')

        #Create the colorbar and set some fomratting 
        norm0=norm
        sm0 = plt.cm.ScalarMappable(cmap=mapcolor, norm=norm0)
        sm0.set_array([])
        cbar0 = fig.colorbar(sm0, ax=ax, label=lab,shrink=0.4,orientation='horizontal',pad=-0.1,anchor=(0.25, 1))
        cbar0.set_label(lab, fontsize=8)
        cbar_tick_labels = cbar0.ax.get_yticklabels()
        for label in cbar_tick_labels:
            label.set_fontsize(8)

        #Formate the first figure
        ax.set_aspect('equal')
        ax.set_title(title+"\n{}".format(year),fontsize=12)
        ax.set_xlabel("Longitude (EPSG 3347, in meters )",fontsize=10)
        ax.set_ylabel("Lattitude (EPSG 3347, in meters)",fontsize=10)
        ax.legend(fontsize=10)
        ax.tick_params(axis='both', which='major', labelsize=8)
        ax.set_xlim([5800000, 7600000])

    #Generate the second subplot, a date vs. count of fires per year
    ax1 = plt.subplot(gs[1, :])
    ax1.bar(all_fires_counted['year_month'].astype(str),all_fires_counted['count'])

    #Get the year and number of fires to plot in red
    fire_month = all_fires_counted[all_fires_counted['year_month'].astype(str)==year]
    print(file)
    print(fire_month)
    print(year)

    #Format sub figure
    ax1.tick_params(axis='both', which='major', labelsize=8)
    ax1.set_title("Forest Fire Counts Per Month from 2001 to 2021",fontsize=12)
    ax1.set_xlabel("Date",fontsize=10)
    ax1.set_ylabel("Count",fontsize=10)
    step = len(all_fires_counted)//20
    ax1.set_xticks(all_fires_counted.index[::step],all_fires_counted['year_month'].astype(str).iloc[::step],rotation=45)
    
    #Plot the red bar if there are fires
    if len(fire_month) != 0:
        ax1.bar(fire_month['year_month'].astype(str),fire_month['count'],color='red')
    
    #Save and show figures
    name = str((int(file.split("_")[0])%100)*12-12 + int(file.split("_")[1]))
    plt.savefig(r"I:\CME538 Project\processed data ontario only\250m\Approximations\combined\\"+name+"_"+".png",dpi = 300)
    plt.show()